In [17]:
import torch
import torch.nn as nn
from torch import tensor
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from datasets import load_dataset
from torchvision.transforms import v2

In [39]:
trainset = load_dataset('Maysee/tiny-imagenet', split='train')
testset = load_dataset('Maysee/tiny-imagenet', split='valid')

In [40]:
trainset.set_format(type="torch", columns=["image", "label"])
testset.set_format(type="torch", columns=["image", "label"])

In [66]:
idx_odd_shape = []
for i in range(len(trainset)):
    shape = trainset[i]['image'].shape
    if shape == torch.Size([64, 64]):
        idx_odd_shape.append(i)

In [54]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,shuffle=True, num_workers=0)

In [57]:
for i, data in enumerate(trainloader, 0):
    image = data['image']
    print(image)

tensor([[[[101, 142, 208],
          [101, 142, 208],
          [101, 142, 208],
          ...,
          [154, 187, 238],
          [154, 188, 236],
          [155, 189, 237]],

         [[102, 143, 209],
          [102, 143, 209],
          [102, 143, 209],
          ...,
          [155, 188, 239],
          [156, 190, 238],
          [157, 191, 239]],

         [[103, 144, 210],
          [103, 144, 210],
          [103, 144, 210],
          ...,
          [157, 191, 239],
          [158, 192, 240],
          [159, 193, 241]],

         ...,

         [[ 82,  80,  81],
          [ 81,  80,  78],
          [ 81,  79,  80],
          ...,
          [101, 102,  96],
          [ 97,  98,  92],
          [ 92,  95,  88]],

         [[ 79,  79,  77],
          [ 79,  80,  75],
          [ 81,  81,  79],
          ...,
          [101, 100,  95],
          [106, 105, 100],
          [108, 109, 101]],

         [[ 86,  87,  82],
          [ 87,  88,  83],
          [ 89,  90,  85],
         

RuntimeError: stack expects each tensor to be equal size, but got [64, 64, 3] at entry 0 and [64, 64] at entry 2

## Choose a Device

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  

In [53]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_layer_one = nn.Conv2d(in_channels=3,out_channels=1024,kernel_size=3 , stride=1, padding=1)
        self.conv_layer_two = nn.Conv2d(in_channels=1024,out_channels=720,kernel_size=3, stride=1, padding=1)
        self.conv_layer_three = nn.Conv2d(in_channels=720,out_channels=440,kernel_size=3, stride=1, padding=1)
        self.conv_layer_four = nn.Conv2d(in_channels=440,out_channels=200,kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(3200,2480)
        self.fc2 = nn.Linear(2480,1680)
        self.fc3 = nn.Linear(1680,980)
        self.fc4 = nn.Linear(980,640)
        
    def forward(self, x):
        x = self.conv_layer_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x,(2,2))
        x = self.conv_layer_two(x)
        x = F.relu(x)
        x = F.avg_pool2d(x,(2,2))
        x = self.conv_layer_three(x)
        x = F.relu(x)
        x = F.max_pool2d(x,(2,2))
        x = self.conv_layer_four(x)
        x = F.relu(x)
        x = F.max_pool2d(x,(2,2))
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x
    

net = Net()     # Create the network instance.
net.to(device)  # Move the network parameters to the specified device.

(100000, 2)

## Choose Loss Function

In [ ]:
loss_func = nn.CrossEntropyLoss()  
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 

In [ ]:
avg_losses = []   # Avg. losses.
epochs = 25       # Total epochs.
print_freq = 10000  # Print frequency.

for epoch in range(epochs):  # Loop over the dataset multiple times.
    running_loss = 0.0       # Initialize running loss.
    for i, data in enumerate(trainloader, 0):
        # Get the inputs.
        inputs, labels = data
        
        # Move the inputs to the specified device.
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients.
        opt.zero_grad()

        # Forward step.
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        
        # Backward step.
        loss.backward()
        
        # Optimization step (update the parameters).
        opt.step()

        # Print statistics.
        running_loss += loss.item()
        if i % print_freq == print_freq - 1: # Print every several mini-batches.
            avg_loss = running_loss / print_freq
            print('[epoch: {}, i: {:5d}] avg mini-batch loss: {:.3f}'.format(
                epoch, i, avg_loss))
            avg_losses.append(avg_loss)
            running_loss = 0.0

print('Finished Training.')

## Plot Training Loss

In [ ]:
plt.plot(avg_losses)
plt.xlabel('mini-batch index / {}'.format(print_freq))
plt.ylabel('avg. mini-batch loss')
plt.show()

## Evaluate on Test Set

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        print("predicted")
        print(predicted)
        print("labels")
        print(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))